In [ ]:
# Notebook vorbereiten
from common.functions import Notebook_vorbereiten
common = Notebook_vorbereiten()
ab_ag_raw = common.load_data("ab_ag_raw")
columns = common.load_data("columns")
uniprot_data = common.load_data("uniprot_data")

#War Strg+Shift+P --> Preferences: Open settings --> jupyter.notebookFileRoot richtig für das Problem mit dem Working Directory?

In [ ]:
# Das was Pauli mit ab_ag in dem alten Repo gemacht hat:

ab_ag = ab_ag_raw[["pdb", "antigen_name", "antigen_species", "Hchain", "Lchain", "antigen_chain"]]

db_count=ab_ag['pdb'].value_counts()
print(pdb_count.max())
print(pdb_count)

print("Anzahl fehlender Werte pro Spalte:")
print(ab_ag.isnull().sum())

# 1. Kopie des DataFrames erstellen
ab_ag_neu = ab_ag.copy()

# 2. Fehlende Werte in der Kopie entfernen
ab_ag_neu.dropna(subset=['antigen_name'], inplace=True)

# 3. Überprüfung
print("Anzahl fehlender Werte nach dem Löschen:")
print(ab_ag_neu['antigen_name'].isnull().sum())
print("Form des neuen DataFrames:", ab_ag_neu.shape)

# 4. Duplikate entfernen in pdb
ab_ag_neu1 = ab_ag_neu.drop_duplicates(subset=['pdb'], keep='first')

print(ab_ag_neu1['antigen_name'].value_counts()[['hemagglutinin','guanine nucleotide-binding protein g(i) subunit alpha-1 | guanine nucleotide-binding protein g(i)/g(s)/g(t) subunitbeta-1 ','spike protein s1']])

